In [3]:
# Train an MLP with 2 inputs, 3-4+ hidden units and one output on the
# following examples (XOR function):
# ((0, 0), 0)
# ((0, 1), 1)
# ((1, 0), 1)
# ((1, 1), 0)

import numpy as np
import matplotlib.pyplot as plt
# from NeuralNetwork import TwoLayerNeural, epoch_learn
from MLP import MultiLayerPerceptron
from main import plot

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

network = MultiLayerPerceptron(2, 10, 1)
network.randomise()

accuracy = []
loss = []
for i in range(1000):
    error = network.train(X, y, 0.95)
    accuracy.append(network.accuracy(X, y))
    loss.append(error)

# plot(accuracy, loss)
y_pred = np.argmax(network.forward(X), axis=1)
print(y_pred)
print(network.forward(X))



[0 0 0 0]
[[0.15134894]
 [0.71715115]
 [0.61119722]
 [0.63205154]]
